In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer



In [2]:
data_path = "../../Dataset_analyse.csv"
df = pd.read_csv(data_path)

df = df.dropna()

In [3]:
X = df.drop('first_week', axis=1)
y = df.first_week

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42)

num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

onehotscale_pipeline = make_pipeline(OneHotEncoder(), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)

polyscale_pipeline = make_pipeline(PolynomialFeatures(2))

my_final_pipeline = make_pipeline(preprocessing, polyscale_pipeline)
my_final_pipeline.fit(X_train)

feature_names = my_final_pipeline.get_feature_names_out(X.columns)

model = make_pipeline(
    my_final_pipeline,
    Lasso(random_state=42, max_iter=100000)
)

param_grid = {'lasso__alpha': [15]}


model = GridSearchCV(model, param_grid,cv =5)
model.fit(X_train, y_train)
grid_score = model.score(X_train, y_train)

best_alpha = model.best_params_['lasso__alpha']

In [ ]:
y_pred_train = model.predict(X_train)


residus =  y_pred_train - y_train


weights = np.exp(-abs(residus) / residus.std())
model.fit(X_train, y_train, lasso__sample_weight=weights)




y_pred = model.predict(X_test)

In [ ]:

r2_cleaned = r2_score(y_test, y_pred)
mse_cleaned = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle après suppression des valeurs extrêmes:")
print(model.best_params_)
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")